In [74]:
import numpy as np
import shap
import pandas as pd
from sklearn.linear_model import LogisticRegression
import sys
sys.path.append('../GilesCode/')
from helper import *
from helper2 import *
from helper2_dep import *
from helper2_indep import *
from helper2_shapley_sampling import *
from helper4_kshap import *
from os.path import join
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

### Logistic dataset

In [75]:
# d = 10
# FEATURE_MEANS = np.repeat(0, d)
# FEATURE_VARS = np.repeat(1, d)
# FEATURE_COVS = [0.5, 0.25]
# COV_MAT = make_banded_cov(FEATURE_VARS, FEATURE_COVS)

# # Randomly generate samples
# np.random.seed(1)
# X = np.random.multivariate_normal(FEATURE_MEANS, COV_MAT, size=10000)
# X_train, X_test = X[:8000], X[8000:]
# xloc = X_test[10].reshape((1,d))

# np.random.seed(1)
# BETA = np.random.normal(0, 1, size = d)
# def model(x):
#     yhat = sigmoid(np.dot(x, BETA))
#     return yhat.item() if x.shape[0]==1 else yhat
# y = (model(X) > 0.5).astype(int)
# y_train, y_test = y[:8000], y[8000:]
# mapping_dict = None

### Census dataset

In [76]:
# X, y = shap.datasets.adult()
# X_display, y_display = shap.datasets.adult(display=True)
# X_binarized = pd.get_dummies(X_display)

# mapping_dict = {}
# for i, col in enumerate(X_display.columns):
#     bin_cols = []
#     for j, bin_col in enumerate(X_binarized.columns):
#         if bin_col.startswith(col):
#             bin_cols.append(j)
#     mapping_dict[i] = bin_cols

# X_norm = (X_binarized-X_binarized.min())/(X_binarized.max()-X_binarized.min())
# y_int = y_display.astype("int8")

# # Split into training and test sets
# np.random.seed(1)
# n, d = X_norm.shape
# n_train = round(n*0.75)
# train_idx = np.random.choice(n, size=n_train, replace=False)
# X_train_pd, y_train = X_norm.iloc[train_idx], y_int[train_idx]
# X_train = X_train_pd.to_numpy()

# test_idx = np.setdiff1d(list(range(n)), train_idx)
# X_test_pd, y_test = X_norm.iloc[test_idx], y_int[test_idx]
# X_test = X_test_pd.to_numpy()


### Bank dataset

In [77]:
# dirpath = "../Data/bank"
# # dirpath = /PATH/TO/DATA
# df_orig = pd.read_csv(join(dirpath, "df_orig.csv"))

# X_train_raw = np.load(join(dirpath, "X_train.npy"))
# X_test_raw = np.load(join(dirpath, "X_test.npy"))
# y_train = np.load(join(dirpath, "Y_train.npy"))
# y_test = np.load(join(dirpath, "Y_test.npy"))
# full_dim = X_train_raw.shape[1] # dimension including all binarized categorical columns
# X_df = pd.read_csv(join(dirpath, "X_df.csv"))


# trainmean, trainstd = np.mean(X_train_raw, axis=0), np.std(X_train_raw, axis=0)
# def rescale(x, trainmean, trainstd):
#     return (x - trainmean) / trainstd
# X_train = rescale(X_train_raw, trainmean, trainstd)
# X_test = rescale(X_test_raw, trainmean, trainstd)

# feature_means = np.mean(X_train, axis=0)
# cov_mat = np.cov(X_train, rowvar=False)


# df_orig.columns = df_orig.columns.str.replace(' ', '_')
# categorical_cols = ['Job', 'Marital', 'Education', 'Default', 'Housing',
#                     'Loan', 'Contact', 'Month', 'Prev_Outcome']
# mapping_dict = get_mapping_dict(df_orig, X_df, X_train_raw, categorical_cols)
# mapping_dict

# d = X_df.shape[1]


### German Credit 

In [78]:
# import sage
# df = sage.datasets.credit()
# # Property, other installment, housing, job, status of checking act, credit history, purpose, savings, employment since, marital status, old debtors
# n = df.shape[0]
# X_df = df.drop(["Good Customer"], axis=1)
# y = df["Good Customer"]

# categorical_columns = [
#     'Checking Status', 'Credit History', 'Purpose', #'Credit Amount', # It's listed but has 923 unique values
#     'Savings Account/Bonds', 'Employment Since', 'Personal Status',
#     'Debtors/Guarantors', 'Property Type', 'Other Installment Plans',
#     'Housing Ownership', 'Job', #'Telephone', 'Foreign Worker' # These are just binary
# ]
# X_binarized = pd.get_dummies(X_df, columns=categorical_columns)
# d_bin = X_binarized.shape[1]

# mapping_dict = {}
# for i, col in enumerate(X_df.columns):
#     bin_cols = []
#     for j, bin_col in enumerate(X_binarized.columns):
#         if bin_col.startswith(col):
#             bin_cols.append(j)
#     mapping_dict[i] = bin_cols

# np.random.seed(1)
# X_norm = (X_binarized-X_binarized.min())/(X_binarized.max()-X_binarized.min())
# n_train = round(n*0.8)
# train_idx = np.random.choice(n, n_train, replace=False)
# X_train, y_train = X_norm.iloc[train_idx].to_numpy(), y.iloc[train_idx].to_numpy()
# test_idx = np.setdiff1d(np.arange(n),train_idx)
# X_test, y_test = X_norm.iloc[test_idx].to_numpy(), y.iloc[test_idx].to_numpy()
# d = X_train.shape[1] # dimension of binarized data

### BRCA

In [79]:
# Load data
np.random.seed(1)
data = pd.read_csv('../Data/brca_small.csv')
X = data.values[:, :-1][:,:20]
Y = data.values[:, -1]
Y = (Y==2).astype(int) # For multiclass problem. May not be predictive. Splits into 2

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=100, random_state=0)
# X_train, X_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=100, random_state=1)

# Normalize
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
# X_val = (X_val - mean) / std
X_test = (X_test - mean) / std
d = X_train.shape[1]
mapping_dict = None

In [80]:
sds = []
for i in range(d):
    uu = np.unique(X_train[:,i])
    if len(uu) == 2:
        sds.append(uu)
    else:
        sds.append(np.repeat(np.std(X_train[:,i]),2))
sds = np.array(sds)

feature_means = np.mean(X_train, axis=0)
cov_mat = np.cov(X_train, rowvar=False)
# Recondition covariance
u, s, vh = np.linalg.svd(cov_mat, full_matrices=True)
K = 10000
if np.max(s)/np.min(s) < K:
    cov2 = cov_mat
else:
    s_max = s[0]
    min_acceptable = s_max/K
    s2 = np.copy(s)
    s2[s <= min_acceptable] = min_acceptable
    cov2 = np.matmul(u, np.matmul(np.diag(s2), vh))


In [81]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler

class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TwoLayerNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.tanh = nn.Tanh()
        self.linear2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out = self.linear1(x)
        out = self.tanh(out)
        out = self.linear2(out)
        out = self.softmax(out)
        return out

# Convert the input and label data to PyTorch tensors
inputs = torch.tensor(X_train, dtype=torch.float32)
labels = torch.tensor(y_train, dtype=torch.long)

# Compute the class weights
class_counts = torch.bincount(labels)
num_samples = len(labels)
class_weights = 1.0 / class_counts.float()
sample_weights = class_weights[labels]

# Create a sampler with balanced weights
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=num_samples, replacement=True)

# Create a DataLoader with the sampler
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset, batch_size=32, sampler=sampler)


torch.manual_seed(0)

# Create an instance
net = TwoLayerNet(input_size=d, hidden_size=50, output_size=2)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()#weight=torch.tensor(weights)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)#.01

# Iterate over the training data in batches
num_epochs = 5

# Train the network for the specified number of epochs
for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        # Zero the gradients for this batch
        optimizer.zero_grad()

        # Compute the forward pass of the network
        outputs = net(inputs)

        # Compute the loss for this batch
        loss = criterion(outputs, labels)

        # Compute the gradients of the loss with respect to the parameters
        loss.backward()

        # Update the parameters using the optimizer
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

Epoch 1/5, Loss: 0.6165
Epoch 2/5, Loss: 0.6549
Epoch 3/5, Loss: 0.5868
Epoch 4/5, Loss: 0.5764
Epoch 5/5, Loss: 0.5998


In [82]:
test_tensor = torch.tensor(X_test, dtype=torch.float32)

with torch.no_grad():
    outputs = net(test_tensor)

n_test = X_test.shape[0]
n_positive_preds = torch.sum(np.argmax(outputs, axis=1)==1).item()
print("{}/{} predictions are for positive class; really {}"
    .format(n_positive_preds,n_test, np.sum(y_test==1)))
Y_preds = torch.argmax(outputs, axis=1)
print("Balanced sampling. {}% accuracy".format(round(100*(np.sum(y_test==Y_preds.numpy())/n_test))))

46/100 predictions are for positive class; really 55
Balanced sampling. 73% accuracy


In [83]:
xloc = X_test[0:1]
xloc_torch = torch.tensor(xloc, dtype=torch.float32).requires_grad_(True)
y_pred = net(xloc_torch)[0,1]
y_pred.backward()
gradient = xloc_torch.grad.detach().numpy().reshape((d, 1))

def neural_net(x):
    output = net(x)[0,1] if x.shape[0]==1 else net(x)[:,1]
    return output

def compute_hessian(x):
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    hessian = torch.autograd.functional.hessian(neural_net, x)
    hessian = hessian.reshape((d,d)).detach().numpy()
    return hessian

hessian = compute_hessian(xloc)
def fmodel(x):
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    return neural_net(x).detach().numpy()

In [84]:
# print("Class imbalance: {}".format(100*(max(np.mean(y_test), 1-np.mean(y_test)))))
# print("Estimation accuracy: {}".format(np.mean((nn.predict(X_test) > 0.5)==y_test)*100))


In [85]:
# xloc = X_test[0:1]  
# def fmodel(xloc):
#     return nn.predict_proba(xloc)[:,1]


In [86]:
shap_CV_true_indep = compute_true_shap_cv_indep(xloc, gradient, hessian, feature_means, cov_mat, mapping_dict=mapping_dict)
sum_shap_CV_true = np.sum(shap_CV_true_indep)
avg_CV_empirical = np.mean(f_second_order_approx(fmodel(xloc),X_train, xloc, gradient, hessian))
pred = fmodel(xloc)#[0]
exp_CV_sum_empirical = pred - avg_CV_empirical
print(sum_shap_CV_true)
print(exp_CV_sum_empirical)


-0.34031945
-0.34014217331407215


In [87]:
independent_features = True
obj_ss = cv_shapley_sampling(fmodel, X_train, xloc, 
                        independent_features,
                        gradient, hessian,
                        mapping_dict=mapping_dict,
                        M=100, n_samples_per_perm=10) # M is number of permutations
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_ss

order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(100*(np.maximum(corr_ests, 0)**2)[order])) # Variance reductions


[75. 79. 86. 73. 80. 71. 74. 89. 83. 80. 66. 75. 38. 91. 47. 94. 86.  0.
 56. 53.]


In [88]:
independent_features = True
obj_kshap = cv_kshap(fmodel, X_train, xloc, 
            independent_features,
            gradient, hessian,
            mapping_dict=mapping_dict,var_method="wls",
            M=1000, n_samples_per_perm=10)
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_kshap

order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(100*(np.maximum(corr_ests,0)**2)[order])) # Variance reductions

[90. 83. 86. 90. 90. 84. 88. 72. 77. 87. 84. 84. 81. 90. 84. 87. 83. 86.
 78. 88.]


In [89]:
print(np.argsort(np.abs(final_ests))[::-1])
print(np.argsort(np.abs(vshap_ests_model))[::-1])
print(np.argsort(np.abs(shap_CV_true_indep).reshape(-1))[::-1])

[14  9  5  6 19  8  4 18 11  2 12 13 15  0  7  1  3 16 17 10]
[14  9  6  5  8 19  4 18 11 12  2 10  0 13  7 17  1  3 16 15]
[14  9  5 19  6 18  8  4 13 12  2 11  0 15 16  7  1 17 10  3]


In [90]:
# Prepare for dependent sampling
M_linear = 1000
D_matrices = make_all_lundberg_matrices(M_linear, cov2)

In [91]:
independent_features = False
shap_CV_true_dep = linear_shap_vals(xloc, D_matrices, feature_means, gradient, mapping_dict=mapping_dict)
obj_dep = cv_shapley_sampling(fmodel, X_train, xloc,
                    independent_features,
                    gradient,
                    mapping_dict=mapping_dict,
                    shap_CV_true=shap_CV_true_dep, # Equivalently, can give D_matrices instead
                    M=100,n_samples_per_perm=10,
                    cov_mat=cov2)
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_dep
order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(100*(np.maximum(corr_ests,0)**2)[order])) # Variance reductions

[89. 84. 86. 79. 80. 82. 88. 84. 92. 88. 82. 90. 92. 77. 89. 86. 86. 89.
 91. 88.]


In [92]:
print(np.argsort(np.abs(final_ests))[::-1])
print(np.argsort(np.abs(vshap_ests_model))[::-1])
print(np.argsort(np.abs(shap_CV_true_dep))[::-1])

[14 19  4 18  8  9  5 11  6 12 17  2 15  0  3 13  1 10 16  7]
[14 19  5 18  4  8  9  6 11 17 12  2 13  1  0 15  3  7 16 10]
[14 19 18  9  4  8  5 12 11 13 17  6 16 15  1 10  0  2  3  7]


In [93]:
# Sometimes get negligible variance reductions from bootstrapped covariance, but good ones from WLS
np.random.seed(2)
independent_features = False
obj_kshap_dep = cv_kshap(fmodel, X_train, xloc,
                    independent_features,
                    gradient,
                    mapping_dict=mapping_dict,
                    shap_CV_true=shap_CV_true_dep,
                    M=1000,n_samples_per_perm=10, var_method="wls",
                    cov_mat=cov2)
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_kshap_dep
order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(100*(np.maximum(corr_ests,0)**2)[order])) # Variance reductions

[90. 91. 87. 93. 93. 93. 91. 93. 92. 93. 90. 92. 92. 93. 92. 94. 90. 94.
 93. 92.]


In [94]:
print(np.argsort(np.abs(final_ests))[::-1])
print(np.argsort(np.abs(vshap_ests_model))[::-1])
print(np.argsort(np.abs(shap_CV_true_dep))[::-1])

[14 19 18  9  4  8  5 11  6 12 17  2 15  1  3 10  0 16 13  7]
[14 19  5 18  6 11  9  4  8 12  2  3 10 15 17  0 13  1 16  7]
[14 19 18  9  4  8  5 12 11 13 17  6 16 15  1 10  0  2  3  7]
